<a href="https://colab.research.google.com/github/ebunalabiTH/ea_assignment-6/blob/main/Assignment_6_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENGS 108 Fall 2023 Assignment 6**

*Due November 14, 2023 at 11:59PM on Github*

**Instructors:** George Cybenko

**TAs:** Sunishka Jain, Diksha Bubna, Ganesh Rohit Nirogi , Kushagra Rawat


---

## **Rules and Requirements**


1.   You are only allowed to use Python packages that are explicity imported in
the assignment notebook or are standard (bultin) python libraries like random, os, sys, etc, (Standard Bultin Python libraries will have a Python.org documentation). For this assignment you may use:
  *   [numpy](https://numpy.org/doc/stable/)
  *   [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  *   [scikit-learn](https://scikit-learn.org/stable/)
  *   [matplotlib](https://matplotlib.org/)
  *   [tensorflow](https://www.tensorflow.org/)

2.   All code must be fit into the designated code or text blocks in the assignment notebook. They are indentified by a **TODO** qualifier.

3. For analytical questions that don't require code, type your answer cleanly in Markdown. For help, see the [Google Colab Markdown Guide](https://colab.research.google.com/notebooks/markdown_guide.ipynb).

---


In [1]:
''' Import Statements '''
import os
import random
import numpy as np
import pandas as pd
import csv
import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Creating a Trump Tweet Generator!!!
Finally it's happened. I am allowed to make my dream assignment. Trust me it's the best assignment. People tell me all the time how they wish they could have had an assignment as great as this. You will see why! But first some housekeeping...

# Data Loading
In this assignment we will be using a repository of Donald Trump tweets scrapped from Twitter through June 2020 from [Kaggle](https://www.kaggle.com/datasets/austinreese/trump-tweets) and will use the following code blocks to download the dataset directly to your Google Drive.

## Creating a Kaggle API Token
First we will need to download an API token from Kaggle in order to download the dataset, so our first step is to create a Kaggle account if you don't already have one. (You should have done this in Assignment 4 in case this looks familiar.
1. Create a Kaggle account by following the sign up instructions [here](https://www.kaggle.com/).
2. Log into your Kaggle account and click your account icon on the upper righthand side.
3. Then select **Account** from the dropdown/sidebar menu.
4. Scroll down to the **API** section and select **Create New API Token**.
5. This will download a JSON file called kaggle.json to your Downloads folder on your computer.
6. Now run the following code block and when the **Browse** button appears, click it and select that kaggle.json file.

In [2]:
# Run this code block after creating a Kaggle API token as instructed above.
! pip install -q kaggle
from google.colab import files

# Will ask you to upload kaggle.json file and remove any old ones.
if os.path.exists('kaggle.json'):
  os.remove('kaggle.json')
files.upload()

# Will create the appropriate directory structure
if not os.path.exists('/root/.kaggle'):
  ! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# Also we are going to make a directory called result
if not os.path.exists('/content/results'):
  ! mkdir /content/result

Saving kaggle.json to kaggle.json


## Downloading the Dataset

7. Now we have downloaded our Kaggle credentials we can now download the Trump Tweets Dataset (or any other Kaggle dataset for that matter) directly into our Google Drive.

In [3]:
! kaggle datasets download austinreese/trump-tweets
# Will check to see if the yoga postures zip file has been unzipped and will unzip the file if not.
! unzip trump-tweets.zip

  0% 0.00/6.88M [00:00<?, ?B/s]
100% 6.88M/6.88M [00:00<00:00, 73.8MB/s]
Archive:  trump-tweets.zip
  inflating: realdonaldtrump.csv     
  inflating: trumptweets.csv         


## Loading the Dataset using Pandas
Now let's inspect the trump tweets dataset and see what we have to work with... Brace yourselves.

In [4]:
# Let's load in the two files that we inflated from the Kaggle download. Both realdonaldtrump.csv and trumptweets.csv are the same.
real_donald_trump_df = pd.read_csv('realdonaldtrump.csv')

In [5]:
real_donald_trump_df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


## Problem 1: Pre-Processing the Tweets
As you may have noticed from the dataframe we just loaded, there are some special characters we need to handle. If we are making a tweet or sentence generator, we don't want to mess with special characters like commas or colons or really even captialization. So in the following section you are going to preprocess the data and strip these special characters out.

### Task 1: The Preprocess function
In the following code block complete the preprocess function that will strip or substitute out various special characters or sequences of characters that may not be ideal when training a sentence generator. We will use the built-in re python library to do a number of substitutions, and I have given you a skeleton below, see if you can complete it.

In [7]:
import re

REGEX_SUBS = {
  '\\\\': ' ',  # Use '\\\\' instead of '\\'
  '\n': ' ',
  '&': '',
  'RT ': '',
  '~': '',
  '#': '',
  '!+': '',
  r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)': 'link',  # Use raw string (prefix 'r') here
  '\*': '',  # Use '\*' instead of '[*]'
  r'[@]\w+': 'user',  # Use raw string (prefix 'r') here
  '[:|;]': '',
  r'[\\x]\W+|\d': '',  # Use raw string (prefix 'r') here
  r'[\\x]\w+': '',  # Use raw string (prefix 'r') here
  '    ': ' ',
  '   ': ' ',
  '  ': ' ',
  '@': ' '
}

def preprocess(text):
    # Perform regular expression substitutions
    for pattern, replacement in REGEX_SUBS.items():
        text = re.sub(pattern, replacement, text)

    # Convert everything to lowercase
    text = text.lower()

    return text


### Task 2: Preprocess the dataset.
Now that we have our preprocess function, let's preprocess all the tweets.

In [8]:
#TODO: Run all the Dataframe content through your preprocess function.
real_donald_trump_df['content'] = real_donald_trump_df['content'].apply(preprocess)


In [9]:
real_donald_trump_df['content']

0        be sure to tune in and watch donald trump on l...
1        donald trump will be appearing on the view tom...
2        donald trump reads top ten financial tips on l...
3        new blog post celebrity apprentice finale and ...
4        "my persona will never be that of a wallflower...
                               ...                        
43347    joe biden was a total failure in government. h...
43348    will be interviewed on   seanhannity tonight a...
43349                                                 link
43350                                                 link
43351                                                 link
Name: content, Length: 43352, dtype: object

### Task 3: Tokenize the Data
The next step in every Natural Language Processing task is to tokenize the data, i.e., seperate our words, special characters, etc. into separate unique tokens. We will be using the [Natural Language Tool Kit](https://www.nltk.org/index.html) in python to accomplish this. Study the nltk API docs and see if you can tokenize our data.

In [10]:
from nltk.tokenize import word_tokenize

tokenized = []
# TODO: Tokenize each preprocessed tweet

# Tokenize the data
real_donald_trump_df['tokenized_content'] = real_donald_trump_df['content'].apply(word_tokenize)

# Display the tokenized data
print(real_donald_trump_df['tokenized_content'])

0        [be, sure, to, tune, in, and, watch, donald, t...
1        [donald, trump, will, be, appearing, on, the, ...
2        [donald, trump, reads, top, ten, financial, ti...
3        [new, blog, post, celebrity, apprentice, final...
4        [``, my, persona, will, never, be, that, of, a...
                               ...                        
43347    [joe, biden, was, a, total, failure, in, gover...
43348    [will, be, interviewed, on, seanhannity, tonig...
43349                                               [link]
43350                                               [link]
43351                                               [link]
Name: tokenized_content, Length: 43352, dtype: object


### Task 4: Build a Vocabulary
Now that we have our data tokenized, let's build a vocabulary including beginning and ending of sentence tokens, i.e., \<s>, \</s> for example. At this point let's also add in these beginning and end tokens into each of our data instances.

In [36]:
# Assuming your initial vocabulary is defined as follows:
vocab_keys = {'<s>', '</s>'}
vocabulary = {token: idx for idx, token in enumerate(vocab_keys)}

# Iterate through all tokenized tweets and update the vocabulary
for tokens in real_donald_trump_df['tokenized_content']:
    for token in tokens:
        # Add the token to the vocabulary if it's not already present
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)

# Print the updated vocabulary
print("Updated Vocabulary:", vocabulary)


Updated Vocabulary: {'</s>': 0, '<s>': 1, 'be': 2, 'sure': 3, 'to': 4, 'tune': 5, 'in': 6, 'and': 7, 'watch': 8, 'donald': 9, 'trump': 10, 'on': 11, 'late': 12, 'night': 13, 'with': 14, 'david': 15, 'letterman': 16, 'as': 17, 'he': 18, 'presents': 19, 'the': 20, 'top': 21, 'ten': 22, 'list': 23, 'tonight': 24, 'will': 25, 'appearing': 26, 'view': 27, 'tomorrow': 28, 'morning': 29, 'discuss': 30, 'celebrity': 31, 'apprentice': 32, 'his': 33, 'new': 34, 'book': 35, 'think': 36, 'like': 37, 'a': 38, 'champion': 39, 'reads': 40, 'financial': 41, 'tips': 42, 'show': 43, 'link': 44, '-': 45, 'very': 46, 'funny': 47, 'blog': 48, 'post': 49, 'finale': 50, 'lessons': 51, 'learned': 52, 'along': 53, 'way': 54, '``': 55, 'my': 56, 'persona': 57, 'never': 58, 'that': 59, 'of': 60, 'wallflower': 61, 'i': 62, '’': 63, 'd': 64, 'rather': 65, 'build': 66, 'walls': 67, 'than': 68, 'cling': 69, 'them': 70, "''": 71, '--': 72, 'j.': 73, 'miss': 74, 'usa': 75, 'tara': 76, 'conner': 77, 'not': 78, 'fired':

In [35]:
# Assuming vocabulary is defined and filled with proper indices
vocabulary_inverse = {}
for token, idx in vocabulary.items():
    vocabulary_inverse[idx] = token

# Print the vocabulary inverse
print("Vocabulary Inverse:", vocabulary_inverse)


Vocabulary Inverse: {0: '</s>', 1: '<s>', 2: 'be', 3: 'sure', 4: 'to', 5: 'tune', 6: 'in', 7: 'and', 8: 'watch', 9: 'donald', 10: 'trump', 11: 'on', 12: 'late', 13: 'night', 14: 'with', 15: 'david', 16: 'letterman', 17: 'as', 18: 'he', 19: 'presents', 20: 'the', 21: 'top', 22: 'ten', 23: 'list', 24: 'tonight', 25: 'will', 26: 'appearing', 27: 'view', 28: 'tomorrow', 29: 'morning', 30: 'discuss', 31: 'celebrity', 32: 'apprentice', 33: 'his', 34: 'new', 35: 'book', 36: 'think', 37: 'like', 38: 'a', 39: 'champion', 40: 'reads', 41: 'financial', 42: 'tips', 43: 'show', 44: 'link', 45: '-', 46: 'very', 47: 'funny', 48: 'blog', 49: 'post', 50: 'finale', 51: 'lessons', 52: 'learned', 53: 'along', 54: 'way', 55: '``', 56: 'my', 57: 'persona', 58: 'never', 59: 'that', 60: 'of', 61: 'wallflower', 62: 'i', 63: '’', 64: 'd', 65: 'rather', 66: 'build', 67: 'walls', 68: 'than', 69: 'cling', 70: 'them', 71: "''", 72: '--', 73: 'j.', 74: 'miss', 75: 'usa', 76: 'tara', 77: 'conner', 78: 'not', 79: 'fir

In [12]:
#TODO: Encode training data.
train_data = []


for tokens in real_donald_trump_df['tokenized_content']:
    # Add beginning of sentence token
    tokens = ['<s>'] + tokens
    # Add end of sentence token
    tokens = tokens + ['</s>']

    # Encode tokens using the vocabulary, ignoring tokens not in the vocabulary
    encoded_tokens = [vocabulary.get(token, vocabulary['<s>']) for token in tokens]

    # Append to the training data
    train_data.append(encoded_tokens)

# Display the encoded training data
print(train_data)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
for tokens in real_donald_trump_df['tokenized_content']:
  # Add beginning of sentence token
  tokens = ['<s>'] + tokens
  # Add end of sentence token
  tokens = tokens + ['</s>']
  print (tokens)

In [ ]:
for tokens in real_donald_trump_df['tokenized_content']:
  # Encode tokens using the vocabulary, ignoring tokens not in the vocabulary
  encoded_tokens = [vocabulary.get(token, vocabulary['<s>']) for token in tokens]
  print(encoded_tokens)

In [15]:
print("Vocabulary:", vocabulary)
print("Tokenized Content:", real_donald_trump_df['tokenized_content'])


Vocabulary: {'</s>': 0, '<s>': 1, 'be': 2, 'sure': 3, 'to': 4, 'tune': 5, 'in': 6, 'and': 7, 'watch': 8, 'donald': 9, 'trump': 10, 'on': 11, 'late': 12, 'night': 13, 'with': 14, 'david': 15, 'letterman': 16, 'as': 17, 'he': 18, 'presents': 19, 'the': 20, 'top': 21, 'ten': 22, 'list': 23, 'tonight': 24, 'will': 25, 'appearing': 26, 'view': 27, 'tomorrow': 28, 'morning': 29, 'discuss': 30, 'celebrity': 31, 'apprentice': 32, 'his': 33, 'new': 34, 'book': 35, 'think': 36, 'like': 37, 'a': 38, 'champion': 39, 'reads': 40, 'financial': 41, 'tips': 42, 'show': 43, 'link': 44, '-': 45, 'very': 46, 'funny': 47, 'blog': 48, 'post': 49, 'finale': 50, 'lessons': 51, 'learned': 52, 'along': 53, 'way': 54, '``': 55, 'my': 56, 'persona': 57, 'never': 58, 'that': 59, 'of': 60, 'wallflower': 61, 'i': 62, '’': 63, 'd': 64, 'rather': 65, 'build': 66, 'walls': 67, 'than': 68, 'cling': 69, 'them': 70, "''": 71, '--': 72, 'j.': 73, 'miss': 74, 'usa': 75, 'tara': 76, 'conner': 77, 'not': 78, 'fired': 79, "'v

# Problem 1: N-gram Language Model
In this problem, you will be building a couple n-gram language modeling and see how well just taking pure frequency counts and building a conditional probability distrbution will work.

## Task 1: A 2-gram (Bigram) Model
Recall that our goal in building a language model is to represent the conditional probability $P(w_i | w_{i-1})$ for pairs of words $w_i$ and $w_j$.

### Part A: Frequencies
Go through the encoded Trump tweets and calculate the frequencies of all words as well as all pair of words that appear next to each other in the corpus.

In [16]:
from collections import defaultdict

def make_grams(tweets, n=2):
    grams = []
    for tweet in tweets:
        # Extract n-grams from each tweet
        for i in range(len(tweet) - n + 1):
            grams.append(tuple(tweet[i:i+n]))
    return grams

def calculate_counts(grams):
    counts = defaultdict(int)
    for gram in grams:
        counts[gram] += 1
    return counts

# Assuming 'train_data' is your encoded training data
# You can use the 'train_data' variable from the previous example

# Create bigrams



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
grams = make_grams(train_data)
counts = calculate_counts(grams)

In [ ]:
counts

### Part B: Probabilities
Now from the counts above we will calculate an associated conditional probabilities.

In [ ]:
probs = defaultdict(dict)
#for gram, count in counts.items():
  #TODO: Do something...
x = 0

  # Calculate probabilities for each bigram
for bigram, count in bigram_counts.items():
    # Extract the first and second word from the bigram
    word1, word2 = bigram
    x += 1
    print (x)
    # Calculate the conditional probability P(word2 | word1)
    probability = count / sum(bigram_counts[prev_word, current_word] for prev_word, current_word in bigram_counts if prev_word == word1)

    # Store the probability in the dictionary
    probs[word1][word2] = probability

# Display the calculated probabilities
print(probs)

In [21]:
print (probs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Part C: Make an Bigram Generator Function
Now that we have our probability distrbution, let's make a generator function so that we can generate random Trump tweets using our bigram language model.

In [20]:
def get_next_word(next_word_probs):
  #TODO: Write a function to get the next word based on the previous words conditional probabilities.
  return something

def generate(start_text='', n=10, probs=probs, vocabulary=vocabulary, vocabulary_inverse=vocabulary_inverse):
  # Helper code to create the start text.
  start_text = ['<s>'] + nltk.word_tokenize(preprocess(start_text))

  #TODO: Encode and generate a trump tweet.
  return something

In [41]:
def get_next_word(next_word_probs):
    # Sample the next word based on the conditional probabilities
    words, probabilities = zip(*next_word_probs.items())
    chosen_word = random.choices(words, probabilities)[0]
    return chosen_word

def generate(start_text='', n=10, probs=probs, vocabulary=vocabulary, vocabulary_inverse=vocabulary_inverse):
    # Helper code to create the start text.
    start_text_tokens = ['<s>'] + nltk.word_tokenize(preprocess(start_text))

    # Encode the start text, handling out-of-vocabulary words
    start_text_encoded = [vocabulary.get(token, vocabulary['<s>']) for token in start_text_tokens]

    # Initialize the generated tweet
    generated_tweet = start_text_encoded.copy()

    # Generate the next words based on the bigram model
    for _ in range(n):
        # Get the last two words in the generated tweet
        prev_word, current_word = generated_tweet[-2:]

        # Check if there are conditional probabilities for the current word
        if current_word in probs and len(probs[current_word]) > 0:
            # Get the conditional probabilities for the current word
            next_word_probs = probs[current_word]

            # Sample the next word
            next_word = get_next_word(next_word_probs)

            # Append the next word to the generated tweet
            generated_tweet.append(next_word)
        else:
            # If no conditional probabilities, break the loop
            break

    # Convert indices back to tokens
    generated_tweet_text = [vocabulary_inverse.get(idx, '<s>') for idx in generated_tweet]

    # Join the tokens to form the final generated tweet
    generated_tweet_text = ' '.join(generated_tweet_text)

    return generated_tweet_text

# Example usage:
generated_tweet = generate(start_text="Make America", n=15)
print(generated_tweet)


<s> make america great day to place was one hell . </s>


In [46]:
generate('I can make')

"<s> i can make anything done right . '' </s>"

## **(Bonus)** Task 2: Make a Trigram Model
Using your code from Parts A-C, see if you can build a trigram (3-gram) model that might make tweets a little more logical.

In [47]:
#TODO: Your code goes here.

def generate_trigram(start_text='', n=10, probs=probs, vocabulary=vocabulary, vocabulary_inverse=vocabulary_inverse):
    # Helper code to create the start text.
    start_text_tokens = ['<s>'] + nltk.word_tokenize(preprocess(start_text))

    # Encode the start text, handling out-of-vocabulary words
    start_text_encoded = [vocabulary.get(token, vocabulary['<s>']) for token in start_text_tokens]

    # Initialize the generated tweet
    generated_tweet = start_text_encoded.copy()

    # Generate the next words based on the trigram model
    for _ in range(n):
        # Get the last three words in the generated tweet
        prev_words = generated_tweet[-3:]

        # Check if there are conditional probabilities for the current trigram
        if tuple(prev_words) in probs and len(probs[tuple(prev_words)]) > 0:
            # Get the conditional probabilities for the current trigram
            next_word_probs = probs[tuple(prev_words)]

            # Sample the next word
            next_word = get_next_word(next_word_probs)

            # Append the next word to the generated tweet
            generated_tweet.append(next_word)
        else:
            # If no conditional probabilities, break the loop
            break

    # Convert indices back to tokens using the inverse vocabulary
    generated_tweet_text = [vocabulary_inverse.get(idx, '<s>') for idx in generated_tweet]

    # Join the tokens to form the final generated tweet
    generated_tweet_text = ' '.join(generated_tweet_text)

    return generated_tweet_text

# Example usage:
# Assume you have already calculated trigram probabilities (probs_trigram) similarly to bigram probabilities
# generated_tweet_trigram = generate_trigram(start_text="Make America", n=15, probs=probs_trigram)
# print(generated_tweet_trigram)

In [54]:
generated_tweet_trigram = generate_trigram(start_text="Make America agin by all means", n=15, probs=probs)
print(generated_tweet_trigram)

<s> make america <s> by all means


# Problem 2: Using a Transformer
In this section we are going to leverage a pretrained transformer, i.e., [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), built by the [Hugging Faces Team](https://huggingface.co/gpt2?text=A+long+time+ago%2C). We will also be using their tokenizers because they have been optimized for the language generation task. You should be aware that behind the scenes, their model is using [PyTorch](https://pytorch.org/), the deep learning library built by Facebook, and is quickly becoming more popular than our beloved Tensorflow.

## Task 1: Install and load the GPT2 Model

In [84]:
# Run the following code to install the HuggingFace's transformer python package.
! pip install -U accelerate
! pip install -U transformers

In [56]:
# TODO: Follow the link above and load the GPT2 model as well as the tokenizer.
# NOTE: Replace GPT2Model with GPT2LMHeadModel (this is the model for finetuning use cases.)

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'  # You can choose other GPT-2 variants like 'gpt2-medium', 'gpt2-large', etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


## Task 2: Formatting our Training Data
Upcoming we will be fine-tunning this GPT-2 model on our Trump Tweets, but in order to leverage some of the utility classes built by HuggingFaces, we want to take our preprocessed Trump tweets and place them in a flat text file.

In [ ]:
#TODO: Take our preprocessed trump tweets and write them to a text file
with open('trumpdata.txt', 'w') as text_file:
  #TODO: Do something

In [58]:
 # Assuming real_donald_trump_df is a DataFrame with a 'content' column
preprocessed_trump_tweets = real_donald_trump_df['content'].tolist()

# Specify the path to the output text file
output_file_path = "trumpdata.txt"

# Write the preprocessed tweets to the text file
with open(output_file_path, 'w', encoding='utf-8') as file:
    for tweet in preprocessed_trump_tweets:
        file.write(tweet + "\n")


## Task 4: Fine-tuning the Model
In the following sections, we will complete a couple functions that will allow us to fine-tune the GPT-2 model to our Trump Tweets. I am going to give you a couple of these helper functions, but leave you to write parts of the training function. See the following documentation from Hugging Faces to see the attributes for the [Trainer Class](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer).

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size = 128):
    # Will load and tokenize the data
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    # Helper Function
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

def train(
    train_file_path,
    model,
    tokenizer,
    output_dir,
    overwrite_output_dir,
    num_train_epochs,
    save_steps
    ):
  #TODO: Use the helper functions above to load the dataset and data collector.

  # Don't mess with.
  tokenizer.save_pretrained(output_dir)
  model.save_pretrained(output_dir)

  # Don't mess with.
  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=8,
          num_train_epochs=num_train_epochs,
      )
  #TODO: Use the Trainer class with the necessary parameters to instaniate the trainer
  trainer = Trainer(

  )

  #TODO: Train and save the model using the train and save functions built into the Trainer class.

In [85]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size=128):
    # Will load and tokenize the data
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm=False):
    # Helper Function
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

def train(
    train_file_path,
    model,
    tokenizer,
    output_dir,
    overwrite_output_dir,
    num_train_epochs,
    save_steps
):
    # Load dataset and data collator
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    # Save tokenizer and model
    tokenizer.save_pretrained(output_dir)
    model.save_pretrained(output_dir)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=8,
        num_train_epochs=num_train_epochs,
        save_steps=save_steps,
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    # Train the model
    trainer.train()

    # Save the model after training
    trainer.save_model()



In [ ]:
#TODO: Set necessary parameters, here are some defaults.
train_file_path = "/content/trumpdata.txt"
output_dir = '/content/result'
overwrite_output_dir = False
num_train_epochs = 1
save_steps = 500

In [86]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.35.1', '0.24.1')

In [88]:
#TODO: Use your train function to train the model. It takes about 30 minutes to train in colab.
# Example usage:
train_file_path = "trumpdata.txt"  # Replace with the path to your preprocessed Trump tweets file
output_directory = "fine_tuned_gpt2_model"
num_epochs = 3
save_steps = 1000



In [ ]:
train(
    train_file_path=train_file_path,
    model=model,
    tokenizer=tokenizer,
    output_dir=output_directory,
    overwrite_output_dir=True,
    num_train_epochs=num_epochs,
    save_steps=save_steps,
)

## Task 5: Creating a Tweet Generator
Now that we have our trained model, it's time to generate some Tweets. Since we should have saved our model and tokenizer to an output directory, I've already made some helper functions to load those in. We will focus on our *generate_text* function. The function will take as input some start text like "I am" or "My country is", etc., as well as a max_length parameter which tells the model how much text to generate. Let's Go!

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(sequence, max_length):
    #TODO: Load in the finetuned model and tokenizer

    # Encode our passed sequence
    ids = tokenizer.encode(
        #TODO:Put something here,
        return_tensors='pt'
        )
    #TODO: Use the generate method in our model class to generate output tokens.
    final_outputs = model.generate(
        #TODO: Complete some of the parameters and leave the parameters I've passed in.

        # Parameters you should leave
        do_sample=True,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    # Function to print and decode output.
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
#TODO: Now generate some text!
generate_text('I can make', 30)